In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')

mnist_train_path = 'train.csv'
mnist_test_path = 'test.csv'

mnist_train_df = pd.read_csv(mnist_train_path, header=None).T
mnist_test_df = pd.read_csv(mnist_test_path, header=None).T
last_column = mnist_test_df.columns[-1]
mnist_train_df.rename(columns={last_column:'Class'}, inplace=True)
mnist_test_df.rename(columns={last_column:'Class'}, inplace=True)

IOError: File train.csv does not exist

In [ ]:
index_0s = mnist_train_df['Class'] == 0
index_1s = mnist_train_df['Class'] == 1
index_3s = mnist_train_df['Class'] == 3
index_5s = mnist_train_df['Class'] == 5

mnist_train_0_1_df = mnist_train_df.loc[index_0s | index_1s]
mnist_train_3_5_df = mnist_train_df.loc[index_3s | index_5s]

In [ ]:
index_0s = mnist_test_df['Class'] == 0
index_1s = mnist_test_df['Class'] == 1
index_3s = mnist_test_df['Class'] == 3
index_5s = mnist_test_df['Class'] == 5

mnist_test_0_1_df = mnist_test_df.loc[index_0s | index_1s]
mnist_test_3_5_df = mnist_test_df.loc[index_3s | index_5s]

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

X_0_1 = mnist_train_0_1_df.drop('Class',axis=1)
X_3_5 = mnist_train_3_5_df.drop('Class',axis=1)

y_0_1 = mnist_train_0_1_df['Class']
y_3_5 = mnist_train_3_5_df['Class']

In [ ]:
X_0_1t = mnist_test_0_1_df.drop('Class',axis=1)
X_3_5t = mnist_test_3_5_df.drop('Class',axis=1)

y_0_1t = mnist_test_0_1_df['Class']
y_3_5t = mnist_test_3_5_df['Class']

In [ ]:
# set the training set before the training!!!
X_train = X_0_1
y_train = y_0_1
X_test = X_0_1t
y_test = y_0_1t
Cs = [0, 1]

In [ ]:
mean_all = np.mean(X_train, axis = 0)
mean_all = mean_all.reshape(784,1)
valid_dims = []
for i in xrange(len(mean_all)):
    if mean_all[i] > 1e-05:
        valid_dims.append(i);
# cut the X_train
X_train2 = X_train.as_matrix()[:, valid_dims]

In [ ]:
# My LDA
# problem: 
# 1. compute d-dimensional mean vector for the different classes
# 2. Compute the scatter matrices (in-between-class and within-class)
# 3. Compute eigencector and eigenvalues for the scatter matrices
# 4. Sort by eigenvalues, choose k eigenvectors form W
# 5.  
# Do not use X_std
# Part 0 nomalize first
#def MyLDA(X, Cs)

# parameters:
# input X
X = X_train2
y = y_train

# the number of dimension remain
k = len(valid_dims)

# Part 1 compuate mean
num_class = len(Cs)
di = X.shape[1]
X_means = []
X_cls = []

for i in Cs:
    tempi = X[y == i]
    X_cls.append(tempi)
    X_means.append(np.mean(tempi, axis = 0))

# Part 2 compute Scatter Matrix Sw
Sw = np.zeros([di,di])
for i in range(0, num_class):
    Sw += (len(X_cls[i])-1)*np.cov(X_cls[i].T)

mean_all = np.mean(X, axis = 0)
mean_all = mean_all.reshape(di,1)

Sb = np.zeros((di, di))
for i in range(0, num_class):
    meani = X_means[i]
    meani = meani.reshape(di, 1)
    Sb += len(X_cls[i]) * (meani - mean_all).dot((meani - mean_all).T)

print Sb.shape

# add small value to the diagonal to make in non singular
#Sw2 = Sw + np.eye(784) * 1e-06

# part solving eigenvec and eigenval for Sw^-1 Sb
eig_val, eig_vec = LA.eig(LA.pinv(Sw).dot(Sb))
eig_val = eig_val.real
eig_vec = eig_vec.real

# Part 4 rank and select
pairs = [[eig_val[i], eig_vec[:, i]] for i in range(len(eig_val))]
pairs = sorted(pairs, key = lambda row : row[0], reverse = True)


# recover all the eig_vecs
eig_vecs_sorted = [row[1] for row in pairs]
W = eig_vecs_sorted[0:k]
W = np.matrix(W)

In [ ]:
W = eig_vecs_sorted[0:1]

In [ ]:
# get back to 784 vector
W2 = []
count = 0;
for i in xrange(784):
    if i in valid_dims:
        W2.append(eig_vecs_sorted[0][count])
        count += 1
    else:
        W2.append(0)

eig_vecs_sorted2 = np.zeros([784,784])

In [ ]:
W2 = np.array(W2)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(W2.reshape((28, 28)).T, cmap='gray', interpolation='nearest')
plt.axis("off")
plt.show()